2017/09/07-08

# 自定义一个widget

通信各部件如下

![Widget layer](images/layout.png)

如果要自定义一个widget, 得在前、后端都需要定义 

# 开始一个HelloWorldWidget

## 后端 (python)

### DOMWidget and Widget


定义一个widget类, 可以基于DOMVidget或者是Widgets, 区别在于前者以Dom结构来渲染；后者则以非Dom渲染(比如WebGL)，根据实际的需求来选择继承

### 几个required的props

* _view_name 前端中view的名字
* _view_module 前端中定义的模块名

In [ ]:
import ipywidgets as widgets
from traitlets import Unicode

class HelloWidget(widgets.DOMWidget):
    _view_name = Unicode('HelloView').tag(sync=True)
    _view_module = Unicode('hello').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    value = Unicode('Hello World!').tag(sync=True)

**`sync=True` keyword argument** 处理后端的这个属性值同步到前端

## 前端 (js)

### models and views

Jupyter框架前端部分依赖于Backbone.js, 这是一个经典的MVC前端框架， 定义在后端中的各种widgets会与前端中对应的Models数据同步，在上面的例子中，**`_view_name`** 在Backbone其中的作用就是用于联结view和model

### define the view

定义你的HelloView，从**`DOMWidgetView.extend`**继承，两个required的属性是_view_name和_view_module

In [ ]:
%%javascript
require.undef('hello');
var HelloView = widget.DOMWidgetView.extend({
    //...
});

### render method

接下来最重要的方法 **`render`**,这是自定义widget渲染的组合逻辑的地方（用**`this.$el`**来操作widget默认的div元素）

访问与某个**`view`**实例相关的**`model`**，用**`view`**的**`this.model` props**获取**`get`和`set`** methods与**Backbone**的**`model`**来交互（据手册上提示， 用**`set`**方法要小心，之后得调用**`touch`**方法，以分发这个属性值已经被更改的事件)

In [ ]:
%%javascript
require.undef('hello');
define('hello', ["@jupyter-widgets/base"], function(widget){
    
    var HelloView = widget.DOMWidgetView.extend({
        
         render: function(){ 
            this.$el.text(this.model.get('value')); 
        }
    });
});

### 动态update model <---> view

In [ ]:
%%javascript
require.undef('hello');
define('hello', ["@jupyter-widgets/base"], function(widget){
    
    var HelloView = widget.DOMWidgetView.extend({
        
        render: function(){ 
            this.value_changed();
            this.model.on('change:value', this.value_changed, this);
        },
        
        value_changed: function() {
            this.$el.text(this.model.get('value')); 
            console.log('value has changed!');
        },
    });
    return { HelloView: HelloView}
});

## 测试

你可以点击相应的代码块来查看widget的运行结果

In [ ]:
w = HelloWidget()
w

In [ ]:
w.value = 'hell World'

# 扩展一下Hello World

## 后端

In [ ]:
import ipywidgets as widgets
from traitlets import Unicode, CInt
class SpinnerWidget(widgets.DOMWidget):
    _view_name = Unicode('SpinnerView').tag(sync=True)
    _view_module = Unicode('spinner').tag(sync=True)
    value = CInt(0).tag(sync=True)

## 前端

用**`.spinner`** 创建一个spinner元素， 在`$el`对象上调用其.spinner方法，但注意前后端的_view_name的值保持一致

参考：[jQuery docs for the spinner control](https://jqueryui.com/spinner/) 

In [ ]:
%%javascript
define('spinner', ["@jupyter-widgets/base"], function(widget){
    
    var SpinnerView = widget.DOMWidgetView.extend({
        
        render: function(){ 

            var that = this;
            this.$input = $('<input />');
            this.$el.append(this.$input);
            this.$spinner = this.$input.spinner({
                change: function( event, ui ) {
                    that.handle_spin();
                },
                spin: function( event, ui ) {
                    that.handle_spin();
                }
            });
            
            this.value_changed();
            this.model.on('change:value', this.value_changed, this);
        },
        
        value_changed: function() {
            this.$spinner.spinner('value', this.model.get('value'));
        },
        
        handle_spin: function() {
            this.model.set('value', this.$spinner.spinner('value'));
            this.touch();
        },
    });
    
    return { SpinnerView: SpinnerView }
});

## 测试

In [ ]:
w = SpinnerWidget(value=5)
w

In [ ]:
w.value